# Creación de csv Maestro

## Importar librerias

In [9]:
import pandas as pd
import os

## Cargar rutas estadisticas del Censo

In [5]:
ruta_data = "../../data/processed_data/"
ruta_estadisticas = "../../data/processed_data/preliminar_estadisticas/region/region_total/"

ruta_1 = f"{ruta_estadisticas}estadisticas_total_comunas_1_Tarapacá.csv"
ruta_2 = f"{ruta_estadisticas}estadisticas_total_comunas_2_Antofagasta.csv"
ruta_3 = f"{ruta_estadisticas}estadisticas_total_comunas_3_Atacama.csv"
ruta_4 = f"{ruta_estadisticas}estadisticas_total_comunas_4_Coquimbo.csv"
ruta_5 = f"{ruta_estadisticas}estadisticas_total_comunas_5_Valparaíso.csv"
ruta_6 = f"{ruta_estadisticas}estadisticas_total_comunas_6_Libertador General Bernardo O'Higgins.csv"
ruta_7 = f"{ruta_estadisticas}estadisticas_total_comunas_7_Maule.csv"
ruta_8 = f"{ruta_estadisticas}estadisticas_total_comunas_8_Biobío.csv"
ruta_9 = f"{ruta_estadisticas}estadisticas_total_comunas_9_La Araucanía.csv"
ruta_10 = f"{ruta_estadisticas}estadisticas_total_comunas_10_Los Lagos.csv"
ruta_11 = f"{ruta_estadisticas}estadisticas_total_comunas_11_Aysén del General Carlos Ibáñez del Campo.csv"
ruta_12 = f"{ruta_estadisticas}estadisticas_total_comunas_12_Magallanes y de la Antártica Chilena.csv"
ruta_13 = f"{ruta_estadisticas}estadisticas_total_comunas_13_Metropolitana de Santiago.csv"
ruta_14 = f"{ruta_estadisticas}estadisticas_total_comunas_14_Los Ríos.csv"
ruta_15 = f"{ruta_estadisticas}estadisticas_total_comunas_15_Arica y Parinacota.csv"
ruta_16 = f"{ruta_estadisticas}estadisticas_total_comunas_16_Ñuble.csv"


## Union de todas las estadisticas regionales

In [34]:
rutas = [globals()[f"ruta_{i}"] for i in range(1, 17)]

dfs=[]

for ruta in rutas:
    df_temp = pd.read_csv(ruta)
    dfs.append(df_temp)

df_maestro = pd.concat(dfs, ignore_index=True)

df_maestro['Código comuna'] = df_maestro['Código comuna'].astype(str)

cambio de nombre a columnas y eliminacion de columna

In [35]:
df_maestro.drop(columns=['País o continente de nacimiento'], inplace=True)
df_maestro = df_maestro.rename(columns={
    'poblacion_total': 'total_poblacion',
    'poblacion_migrante': 'total_poblacion_migrante',
    'porcentaje': 'porcentaje_migrante_total'
})

## Se importan los csv
Se debe importar la informacion preprocesada de FONASA, CASEN, red hospitalaria, atenciones y área comunal.

In [37]:
# FONASA
df_fonasa = pd.read_csv(os.path.join(ruta_data, "fonasa.csv"))
df_fonasa = df_fonasa.rename(columns={"PORCENTAJE_EXTRANJEROS": "porcentaje_migrante_fonasa"})
df_fonasa['Código comuna'] = df_fonasa['Código comuna'].astype(str)

# CASEN
df_casen = pd.read_csv(os.path.join(ruta_data, "casen_niveles.csv"))
df_casen['Código comuna'] = df_casen['Código comuna'].astype(str)

# RED HOSPITALARIA
df_red = pd.read_csv(os.path.join(ruta_data, "red_hospitalaria_publica.csv"))
df_red = df_red.rename(columns={"Código Comuna": "Código comuna"})
df_red['Código comuna'] = df_red['Código comuna'].astype(str)

# ATENCIONES
df_atenc = pd.read_csv(os.path.join(ruta_data, "total_atenciones_promedio.csv"))
df_atenc['Código comuna'] = df_atenc['Código comuna'].astype(str)

# ÁREA
df_area = pd.read_csv(os.path.join(ruta_data, "Area_territorial.csv"))
df_area['Código comuna'] = df_area['Código comuna'].astype(str)

## Se unen todos los csv

In [38]:
# Hacemos Left Join para no perder comunas aunque no tengan datos en otras tablas
df_final = df_maestro.merge(df_fonasa[['Código comuna', 'porcentaje_migrante_fonasa']], on='Código comuna', how='left')
df_final = df_final.merge(df_casen[['Código comuna', 'ruralidad', 'nivel_promedio', 'desviacion_estandar']], on='Código comuna', how='left')
df_final = df_final.merge(df_red[['Código comuna', 'Total_centros']], on='Código comuna', how='left')
df_final = df_final.merge(df_atenc[['Código comuna', 'promedio_pacientes', 'porcentaje_migrantes_promedio']], on='Código comuna', how='left')
df_final = df_final.merge(df_area[['Código comuna', 'Area']], on='Código comuna', how='left')

# Rellenar nulos 
df_final['Total_centros'] = df_final['Total_centros'].fillna(0)

## Calculo de tasas

In [39]:
# Tasa de Centros por 10.000 habitantes
df_final['tasa_centros_10k'] = (df_final['Total_centros'] / df_final['total_poblacion']) * 10000

# Tasa de Atenciones por 10.000 habitantes
df_final['tasa_atenciones_10k'] = (df_final['promedio_pacientes'] / df_final['total_poblacion']) * 10000

# Densidad Poblacional (Hab/km2)
df_final['densidad_poblacional'] = df_final['total_poblacion'] / df_final['Area']

# redondear
cols_round = ['tasa_centros_10k', 'tasa_atenciones_10k', 'densidad_poblacional']
df_final[cols_round] = df_final[cols_round].round(2)

## Se crea csv Maestro

In [41]:
salida = os.path.join(ruta_data, "maestro/csv_maestro.csv")
df_final.to_csv(salida, index=False, encoding='utf-8')